In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import sys
sys.path.insert(0,'/content/drive/MyDrive/Skola/py-AI/wildfire/test_obj')

In [3]:
!pip install torchvision

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
%%shell

git clone https://github.com/pytorch/vision.git
cd vision
git checkout v0.8.2

cp references/detection/utils.py ../
cp references/detection/transforms.py ../
cp references/detection/coco_eval.py ../
cp references/detection/engine.py ../
cp references/detection/coco_utils.py ../

Cloning into 'vision'...
remote: Enumerating objects: 287309, done.
remote: Counting objects: 100% (8435/8435), done.
remote: Compressing objects: 100% (526/526), done.
remote: Total 287309 (delta 7983), reused 8309 (delta 7892), pack-reused 278874
Receiving objects: 100% (287309/287309), 575.38 MiB | 41.77 MiB/s, done.
Resolving deltas: 100% (263085/263085), done.
Note: switching to 'v0.8.2'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false

HEAD is now at 2f40a483d7 [v0.8.X] .circleci: Add Python 3.9 to CI (#3063)


In [5]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torchvision.transforms as transforms
import torchvision
import torch

from matplotlib.patches import Rectangle
from matplotlib.lines import Line2D
from PIL import Image
from torchvision.models.detection import FasterRCNN
from torch.utils.data import Dataset, DataLoader
from typing import Any

In [6]:
test_df = pd.read_csv('/content/drive/MyDrive/Skola/py-AI/wildfire/test_obj/test_annotations.csv')
test_images = '/content/drive/MyDrive/Skola/py-AI/wildfire/test_obj/test'

test_df.head()

,filename,width,height,class,xmin,ymin,xmax,ymax
0,ck0t4z1yrkmfv0794t2oqhd3f_jpeg.rf.17653d46c3ee...,640,480,smoke,514,209,616,285
1,ck0ujmglz85u10a468qt0d4fc_jpeg.rf.18d02755e8c9...,640,480,smoke,162,210,635,302
2,ck0uivtpc841h0a46ydgo7566_jpeg.rf.00134176dc29...,640,480,smoke,308,206,392,252
3,ck0nehpd69bax0721onacbe33_jpeg.rf.02ed50fbcb97...,640,480,smoke,274,229,454,311
4,ck0ow7vs07tuz08485n4yz5si_jpeg.rf.179fc0e59422...,640,480,smoke,386,218,607,285


In [7]:
LABELS = test_df['class'].unique()
NUM_OF_CLASSES = len(LABELS)+1
SAVE_PATH = '/content/drive/MyDrive/Skola/py-AI/wildfire/test_obj/models/'
MODEL_NAME = 'model_include_fire.pt'
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
TRANSFORM = transforms.Compose([transforms.ToTensor()])
IMG_PATH = '../data/new-data/datasets/full/fire/fire-2065.jpg'

In [8]:
COLORS = ['#d90166', '#8f00f1', '#d0ff14', '#eb5030', '#ff000d', '#66ff00',
          '#0203e2', '#04d9ff', '#ff00ff', '#fffd01', '#e56024', '#dfff4f',
          '#ff3503', '#6600ff', '#f7b718', '#fe0002', '#45cea2', '#ff85ff',
          '#1974d2', '#fe6700']

In [9]:
class LabelMap:
    def __init__(self, labels: list) -> None:
        self._map = {c: i+1 for i, c in enumerate(labels)}
        self.reversed_map = {i: c for i, c in enumerate(labels)}

    def fit(self, df: pd.DataFrame, col: str) -> pd.DataFrame:
        df[col] = df[col].map(self._map)
        return df


class WildfireDataset(Dataset):
    def __init__(self, df: pd.DataFrame, img_path: str, labels: list, transforms: Any = None, **kwargs) -> None:
        super().__init__(**kwargs)
        self.df = df
        self.img_path = img_path
        self.labels = labels
        self.images = self.df['filename'].unique()
        self.transforms = transforms

    def __len__(self) -> int:
        return len(self.images)

    def __getitem__(self, i: int) -> tuple:
        img_file = os.path.join(self.img_path, self.images[i])

        img = Image.open(img_file)
        #img = img.astype(np.float32)
        #img = img/255.0

        img_data = self.df.loc[self.df['filename'] == self.images[i]]

        xmins = img_data['xmin'].values
        ymins = img_data['ymin'].values
        xmaxs = img_data['xmax'].values
        ymaxs = img_data['ymax'].values

        boxes = torch.as_tensor(np.stack([xmins, ymins, xmaxs, ymaxs], axis=1), dtype=torch.float32)
        labels = torch.as_tensor(img_data['class'].values, dtype=torch.int64)
        _id = torch.tensor([i])

        areas = (boxes[:,3] - boxes[:,1]) * (boxes[:,2] - boxes[:,0])
        areas = torch.as_tensor(areas, dtype=torch.float32)

        iscrowd = torch.zeros((len(labels),), dtype=torch.int64)

        target = dict()
        target['boxes'] = boxes
        target['labels'] = labels
        target['image_id'] = _id
        target['area'] = areas
        target['iscrowd'] = iscrowd

        if self.transforms:
            img = self.transforms(img)
            #img = transformed['image']
            #target['boxes'] = torch.as_tensor(transformed['bboxes'], dtype=torch.float32)
    
        return torch.as_tensor(img, dtype=torch.float32), target

    def get_h_w(self, image: str) -> tuple:
        """Get height and width of image"""
        img_data = self.df.loc[self.df['filename'] == image]
        return img_data['width'].values[0], img_data['height'].values[0]

In [10]:
def encode_label(df: pd.DataFrame, col: str, map_dict: dict) -> pd.DataFrame:
    df[col] = df[col].map(map_dict)
    return df

def collate_fn(batch: tuple) -> tuple:
    return tuple(zip(*batch))

def load_model(model_path: str) -> FasterRCNN:
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(
        pretrained=False,
        num_classes=NUM_OF_CLASSES
    )
    IN_FEATURES = model.roi_heads.box_predictor.cls_score.in_features

    model.load_state_dict(torch.load(model_path))
    model.eval()

    return model

def predict(model, img: Image) -> tuple:
    pred_img = img.view(1, 3, img.shape[1], img.shape[2])

    preds = model(pred_img)
    outputs = [{k: v.to(torch.device('cpu')) for k, v in target.items()} for target in preds]

    boxes = outputs[0]['boxes'].data.cpu().numpy().astype(np.int32)
    scores = outputs[0]['scores'].data.cpu().numpy()
    labels = outputs[0]['labels'].data.cpu().numpy().astype(np.int32)
    
    return boxes, scores, labels

def plot_prediction(img_path: str, predictions: tuple) -> None:
    patches = []

    img = Image.open(img_path)
    _, ax = plt.subplots(figsize=(13,7))
    plt.imshow(img)

    box_counter = 0
    for box, score, label in zip(predictions[0], predictions[1], predictions[2]):
        box_counter += 1
        score *= 100
        label = f'{str(LABELS[label-1])} : {score: .2f}%'

        x_min = int(box[0])
        y_min = int(box[1])
        x_max = int(box[2])
        y_max = int(box[3])

        plt.gca().add_patch(Rectangle(
            (x_min, y_min),
            x_max - x_min,
            y_max - y_min,
            edgecolor=COLORS[box_counter],
            facecolor=None,
            fill=False,
            lw=1
        ))

        patch = Line2D(
            [0], [0],
            marker='o',
            color='w',
            markerfacecolor=COLORS[box_counter],
            label=label
        )
        patches.append(patch)
    
    box = ax.get_position()
    ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])
    ax.legend(
        bbox_to_anchor=(1.05, 1),
        loc='upper left',
        borderaxespad=0.,
        handles=patches
    )

    plt.show()

In [11]:
model = load_model(SAVE_PATH + MODEL_NAME)

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

In [12]:
label_map = LabelMap(LABELS)
test_df = encode_label(test_df, 'class', label_map._map)

In [13]:
test_dataset = WildfireDataset(test_df, test_images, LABELS, TRANSFORM)

# Is DataLoader necessary?
test_dataloader = DataLoader(
    test_dataset,
    batch_size=4,
    shuffle=True,
    num_workers=2,
    collate_fn=collate_fn
)

In [14]:
images, targets = next(iter(test_dataset))
targets

{'boxes': tensor([[514., 209., 616., 285.]]),
 'labels': tensor([1]),
 'image_id': tensor([0]),
 'area': tensor([7752.]),
 'iscrowd': tensor([0])}

In [15]:
MAP_targets, MAP_preds = list(), list()

for image, target in test_dataset:
    #pred_img = TRANSFORM(image)
    prediction = predict(model, image)
    MAP_preds.append(dict(
        boxes=torch.tensor(prediction[0]),
        scores=torch.tensor(prediction[1]),
        labels=torch.tensor(prediction[2])
    ))
    MAP_targets.append(dict(
        boxes=target['boxes'],
        labels=target['labels']
    ))

MAP_preds[2]

{'boxes': tensor([[302, 204, 412, 252]], dtype=torch.int32),
 'scores': tensor([0.9433]),
 'labels': tensor([1], dtype=torch.int32)}

In [16]:
!pip install torchmetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.4/512.4 KB 10.8 MB/s eta 0:00:00


In [17]:
from torchmetrics.detection.mean_ap import MeanAveragePrecision

metric = MeanAveragePrecision()
metric.update(MAP_preds, MAP_targets)

In [18]:
metric.compute()

{'map': tensor(0.3073),
 'map_50': tensor(0.6371),
 'map_75': tensor(0.2525),
 'map_small': tensor(0.2783),
 'map_medium': tensor(0.2809),
 'map_large': tensor(0.3584),
 'mar_1': tensor(0.3083),
 'mar_10': tensor(0.4550),
 'mar_100': tensor(0.4749),
 'mar_small': tensor(0.3556),
 'mar_medium': tensor(0.4582),
 'mar_large': tensor(0.5176),
 'map_per_class': tensor(-1.),
 'mar_100_per_class': tensor(-1.)}

In [27]:
ACC_targets, ACC_preds = list(), list()
for i in range(len(MAP_preds)):
    ACC_preds.append([int(j) for j in MAP_preds[i]['labels'][:len(MAP_targets[i]['labels'])]])
    ACC_targets.append([int(j) for j in MAP_targets[i]['labels']])


In [28]:
ACC_preds

[[1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [2, 2],
 [2, 2, 2, 2, 2],
 [1, 1, 1, 1, 1, 1],
 [2, 2],
 [2, 2, 2, 2],
 [2],
 [2, 2, 2],
 [2],
 [2, 2, 2],
 [2, 1, 2],
 [2, 2, 2, 2],
 [2, 2],
 [2, 2, 2],
 [2, 2, 2],
 [2, 2],
 [2, 2],
 [2, 2, 2, 2, 2, 2],
 [2],
 [2, 1, 1],
 [2],
 [2],
 [1, 1, 1, 1],
 [2, 2, 2, 2, 1],
 [2, 1, 2],
 [2, 2],
 [2, 2, 2],
 [2, 2, 2],
 [2, 2, 2, 2, 1, 1],
 [2, 2],
 [1, 2, 2],
 [2],
 [2, 2, 2, 2],
 [2, 1],
 [2, 2],
 [1, 1, 2, 1]]

In [29]:
from torchmetrics.functional.classification import accuracy

accuracy(ACC_preds, ACC_targets, task='multiclass', num_classes=NUM_OF_CLASSES)

AttributeError: ignored

In [21]:
import pickle

with open('/content/drive/MyDrive/Skola/py-AI/wildfire/test_obj/map_pred.pkl', 'wb') as f:
    pickle.dump(MAP_preds, f)

with open('/content/drive/MyDrive/Skola/py-AI/wildfire/test_obj/map_traget.pkl', 'wb') as f:
    pickle.dump(MAP_targets, f)

In [22]:
with open('/content/drive/MyDrive/Skola/py-AI/wildfire/test_obj/map_pred.pkl', 'rb') as f:
    test_list = pickle.load(f)
test_list

[{'boxes': tensor([[521, 211, 618, 286]], dtype=torch.int32),
  'scores': tensor([0.9930]),
  'labels': tensor([1], dtype=torch.int32)},
 {'boxes': tensor([[169, 240, 488, 292],
          [164, 251, 392, 288],
          [164, 253, 305, 283],
          [210, 234, 623, 303],
          [161, 255, 247, 280],
          [159, 206, 535, 290]], dtype=torch.int32),
  'scores': tensor([0.8945, 0.5829, 0.5777, 0.5639, 0.0923, 0.0811]),
  'labels': tensor([1, 1, 1, 1, 1, 1], dtype=torch.int32)},
 {'boxes': tensor([[302, 204, 412, 252]], dtype=torch.int32),
  'scores': tensor([0.9433]),
  'labels': tensor([1], dtype=torch.int32)},
 {'boxes': tensor([[266, 231, 424, 311]], dtype=torch.int32),
  'scores': tensor([0.9911]),
  'labels': tensor([1], dtype=torch.int32)},
 {'boxes': tensor([[441, 218, 608, 289],
          [523, 242, 614, 289]], dtype=torch.int32),
  'scores': tensor([0.9896, 0.1347]),
  'labels': tensor([1, 1], dtype=torch.int32)},
 {'boxes': tensor([[126, 208, 175, 283]], dtype=torch.int